# RAG (Retrieval-Augmented Generation) 시스템

이 노트북은 LangChain과 Upstage API를 사용하여 RAG 시스템을 구현합니다.

RAG는 다음 단계로 구성됩니다:
1. 문서 로드
2. 문서 분할
3. 임베딩 생성
4. 벡터 데이터베이스 생성
5. 검색기 생성
6. 프롬프트 생성
7. 언어모델 생성
8. 체인 생성 및 실행

## 설정

먼저 Upstage API 키를 설정해주세요.

In [ ]:
# Upstage API 키를 여기에 입력하세요
UPSTAGE_API_KEY = "your_upstage_api_key_here"

## 라이브러리 Import

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage, UpstageEmbeddings

## 단계 1: 문서 로드 (Load Documents)

PDF 파일을 로드합니다. PyMuPDFLoader를 사용하여 PDF 문서를 읽어옵니다.

In [ ]:
loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

print(f"로드된 문서 수: {len(docs)}")
print(f"첫 번째 문서 미리보기: {docs[0].page_content[:200]}...")

## 단계 2: 문서 분할 (Split Documents)

긴 문서를 작은 청크로 분할합니다. 이는 검색 성능을 향상시키고 모델의 컨텍스트 길이 제한을 해결합니다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

print(f"분할된 문서 청크 수: {len(split_documents)}")
print(f"첫 번째 청크 미리보기: {split_documents[0].page_content[:200]}...")

## 단계 3: 임베딩 생성 (Embedding)

Upstage의 임베딩 모델을 사용하여 텍스트를 벡터로 변환합니다.

In [ ]:
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="embedding-query"
)

print("임베딩 모델이 성공적으로 초기화되었습니다.")

## 단계 4: 벡터 데이터베이스 생성 및 검색기 설정

FAISS를 사용하여 벡터 데이터베이스를 생성하고 검색기를 설정합니다.

In [ ]:
# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 검색기 생성
retriever = vectorstore.as_retriever()

print("벡터 데이터베이스와 검색기가 성공적으로 생성되었습니다.")

## 단계 5: 프롬프트 생성 (Create Prompt)

질문 답변을 위한 프롬프트 템플릿을 생성합니다.

In [ ]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

print("프롬프트 템플릿이 생성되었습니다.")

## 단계 6: 언어모델 생성 (LLM)

Upstage의 Solar Pro 모델을 사용하여 언어모델을 생성합니다.

In [ ]:
llm = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    reasoning_effort="high"
)

print("언어모델이 성공적으로 초기화되었습니다.")

## 단계 7: 체인 생성 (Create Chain)

검색기, 프롬프트, 언어모델을 연결하여 RAG 체인을 생성합니다.

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG 체인이 성공적으로 생성되었습니다.")

## 단계 8: 질문 실행 (Run Chain)

문서에 대한 질의를 입력하고 답변을 받습니다.

In [ ]:
# 질문 설정
question = "삼성전자가 자체 개발한 AI 의 이름은?"

# 체인 실행
response = chain.invoke(question)

print(f"질문: {question}")
print(f"답변: {response}")

## 추가 질문 테스트

다른 질문들도 테스트해볼 수 있습니다.

In [ ]:
# 새로운 질문으로 테스트
new_question = "AI 기술의 최신 동향은 무엇인가요?"
new_response = chain.invoke(new_question)

print(f"질문: {new_question}")
print(f"답변: {new_response}")